In [2]:
#| include: false

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#| include: false
%load_ext rpy2.ipython

## 4.
15 pts Imagine that you are in a meeting to devise a statistical analysis plan for an upcoming cohort study of N = 3,000 subjects (to be sampled independently) in which the primary aim is to compare some continuous outcome, Y , between two groups defined by an exposure status. The exposure occurs in about one-third of individuals in the population. Is is suspected a priori that the outcome variance in the exposed group will be substantively larger in the exposed group as compared to the unexposed group (due to, e.g., heterogeneous response to the exposure). Despite this, your collaborator suggests Student’s t-test (i.e., the t-test that assumes equal variances between groups) in order test the hypothesis of a nonzero mean difference, as this has been standard in the literature. As the group’s biostatistician, you perform a simulation to illustrate the possible pitfalls of this approach. The R function to conduct the simulation is shown below (mind the line numbers on the left-hand side, as questions (a)-(e) below refer to them):

In [4]:
%%R
## add notation for the R code
simulation <- function(N, p.exp, sigma0, k, nsim, seed) {
set.seed(seed) # set seed
results <- matrix(0, ncol = 1, nrow = nsim) # initialize results
for (j in 1:nsim) { # loop over nsim
x <- rbinom(n = N, size = 1, prob = p.exp) # generate x
N1 <- sum(x) # number of treated
N0 <- N - sum(x) # number of control
y0 <- rnorm(n = N0, mean = 0, sd = sigma0) # generate y0
y1 <- rnorm(n = N1, mean = 0, sd = sqrt(k * sigma0^2)) # generate y1
p <- t.test(x = y0, y = y1, var.eq = TRUE)$p.value # t-test
results[j, 1] <- as.numeric(p < 0.05) # store results
}
return(colMeans(results)) # return the average
}

simulation(N=3000, p.exp=1/3,  sigma0=10, k=2, nsim=50000, seed=2021)

[1] 0.07926


(a) Briefly explain the major purpose of the seed argument (referred to on Line 3).

(b) Briefly explain what Lines 7-9 of the simulation code are doing.

(c) Briefly explain the role of the k argument (referred to on Line 11).

(d) In the language of hypothesis testing, what does the number “0.05” refer to on Line 13?

(e) In the language of hypothesis testing, what is the value returned by the function (Line 15)?

(f) Briefly summarize the main result of the simulation study and highlight the importance of its implica- tions.

(g) What alternative test would you suggest to answer your collaborator’s scientific question?

(h) Illustrate the advantage of the approach you identified in part (g) by re-running the simulation study under the same setup, but making a single key modification to the function. Specifically state the change you’ve made, and briefly summarize your findings.

In [5]:
%%R
## add notation for the R code
simulation <- function(N, p.exp, sigma0, k, nsim, seed) {
set.seed(seed) # set seed
results <- matrix(0, ncol = 1, nrow = nsim) # initialize results
for (j in 1:nsim) { # loop over nsim
x <- rbinom(n = N, size = 1, prob = p.exp) # generate x
N1 <- sum(x) # number of treated
N0 <- N - sum(x) # number of control
y0 <- rnorm(n = N0, mean = 0, sd = sigma0) # generate y0
y1 <- rnorm(n = N1, mean = 0, sd = sqrt(k * sigma0^2)) # generate y1
p <- t.test(x = y0, y = y1, var.eq = F)$p.value # t-test unequal variance
results[j, 1] <- as.numeric(p < 0.05) # store results
}
return(colMeans(results)) # return the average
}

simulation(N=3000, p.exp=1/3,  sigma0=10, k=2, nsim=50000, seed=2021)

[1] 0.04986


## 5.
50 pts Suppose you are working with a team that is interested in understanding the link between recreational drug use and subsequent visits to the emergency room (ER). They collect data on N = 400 independently sampled individuals and survey them on their history of drug use. If a subject had an ER visit within five years of study enrollment, it was noted (data set: er.csv). The variables in the data set are as follows:

In [6]:
er = pd.read_csv("./er.csv")
er.head()

,agegrp,tob,alc,mrj,illeg,er
0,3,0,1,0,0,0
1,0,0,1,1,1,0
2,3,0,0,0,0,1
3,1,0,1,0,0,0
4,1,0,1,0,1,0


(a) Briefly summarize the distribution of variables in the data set. You may include tables and figures if you choose, but your response should not exceed one double-spaced page (tables and figures included).

In [13]:
#| echo: false
### make 2x2 table of every variable vs er
def make_table(df, var):
    df = df[["er", var]]
    df = df.groupby(["er", var]).size().reset_index(name='counts')
    df = df.pivot(index='er', columns=var, values='counts')
    df = df.fillna(0)
    df = df.astype(int)
    return df

for cname in er.columns:
    if cname != "er":
        print(make_table(er, cname))
        print()

agegrp   0   1   2   3   4
er                        
0       79  58  51  68  48
1       22  26  11  17  20

tob    0   1
er          
0    239  65
1     67  29

alc    0    1
er           
0    153  151
1     57   39

mrj    0   1
er          
0    273  31
1     77  19

illeg    0   1
er            
0      286  18
1       95   1



(b) Perform a principled analysis in which you investigate the association between recreational drug use and ER visits. When writing your response:

• Remember that this is an open-ended question in which there are multiple defensible ways to respond. Please select and implement only one approach (that may or may not involve results from more than one model).

• Describe the method you are implementing such that it could essentially be replicated without having to look at your code.

• Clearly state and briefly defend any choices you make in your analysis.

• Present and interpret your results using suitable measures of association that could be understand-
able to a scientific collaborator.

• You may include tables and/or figures if you believe they will aid the presentation of your response.

• Concede at least one limitation of the approach you choose.

• Your response should be no longer than two double-spaced pages (including tables and figures, but
excluding code.

• Please present code only as an appendix.

In [25]:
#| echo: false
# Perform a anova test on drugs

# build a logistic model on er vs tob+alc+mrj+illeg
from statsmodels.formula.api import logit

model = logit("er ~ tob + alc + mrj + illeg", data=er).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.522541
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                     er   No. Observations:                  400
Model:                          Logit   Df Residuals:                      395
Method:                           MLE   Df Model:                            4
Date:                Thu, 11 May 2023   Pseudo R-squ.:                 0.05179
Time:                        13:04:52   Log-Likelihood:                -209.02
converged:                       True   LL-Null:                       -220.43
Covariance Type:            nonrobust   LLR p-value:                 0.0001368
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.1091      0.164     -6.772      0.000      -1.430      -0.788
tob            0.5126      0.

Based on the F-statistcs, which is anova test for the null hypothesis that at least one of the coefficients is nonzero, we can reject the null hypothesis that all coefficients are zero. This means that at least one of the coefficients is nonzero.

(c) Perform a principled analysis in which you evaluate the overall out-of-sample predictive ability of recre- ational drug use as predictors for ER visits. When writing your response:


In [21]:
#| echo: false
# create data partition
from sklearn.model_selection import train_test_split

X = er[["tob", "alc", "mrj", "illeg"]]
y = er["er"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123123)


In [30]:
#| echo: false
# build a logistic model on the training set
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X_train, y_train)

# predict on the test set and compute accuracy
y_pred = model.predict(X_test)
print("Accuracy:", np.mean(y_pred == y_test))


Accuracy: 0.755
